# **Demers_CS664_Project: Pandemic Simulator**

Note: This is loosely based on the "Wumpus World" example given in the book and related web resources. The idea is that there is a pandemic which begins with one infected individual ("Patient Zero"). This agent can infect other agents (the "Residents") which can in turn affect others (including the player). The goal is to find the "cure" and get it to the "hospital" before your health level falls to 0 and the game ends.

In [0]:
#!pip install IPythonblocks

In [0]:
#Reference:[6]  
#!pip install pydrive                          

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth                    
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()                         
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
your_module = drive.CreateFile({"id": "1muAD28yRZPEdHUojc-i80aQRbMsKTqAz"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("utils.py")          # Save the .py module file to Colab VM

import utils      

In [0]:
from utils import remove_all, unique, first, probability, isnumber, issequence, Expr, expr, subexpressions, extend, distance_squared, turn_heading

In [0]:
from statistics import mean
from ipythonblocks import BlockGrid
from IPython.display import HTML, display, clear_output
from time import sleep
import random
import copy
import collections
import numbers
import heapq
import itertools
import random
from collections import defaultdict, Counter, deque
import networkx as nx
import sys

#from csp import parse_neighbors, UniversalDict

## **Setup environment**

Implement "things" which exist in the "world":

In [0]:
#Reference:[1B] 
# from book code:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # We need this?
        pass

Create a generic Agent derived from Thing:

In [0]:
#Reference:[1B] 
# a general agent
class Agent(Thing):
    """An Agent is a subclass of Thing with one required slot,
    .program, which should hold a function that takes one argument, the
    percept, and returns an action. (What counts as a percept or action
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method,
    then the program could 'cheat' and look at aspects of the agent.
    It's not supposed to do that: the program can only look at the
    percepts. An agent program that needs a model of the world (and of
    the agent itself) will have to build and maintain its own model.
    There is an optional slot, .performance, which is a number giving
    the performance measure of the agent in its environment."""

    #added lealth level:
    def __init__(self, program=None):
        self.alive = True
        self.sick = False
        self.bump = False
        self.health = 100
        self.holding = []
        self.performance = 100
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))
#MD: This is the important part that makes the agent "alive"
            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False

def TraceAgent(agent):
    """Wrap the agent's program to print its input and output. This will let
    you see what the agent is doing in the environment."""
    old_program = agent.program

    def new_program(percept):
        action = old_program(percept)
        print('{} perceives {} and does {}'.format(agent, percept, action))
        return action

    agent.program = new_program
    return agent

Set up environment superclass:

In [0]:
#Reference:[1B] 
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        
        for step in range(steps):
            if self.is_done():
                return
            self.step()
            #MD: TODO: Troubleshooting step: (MD: uncommenting for now 3/7)
            print("Current status:")
            print(self.env.to_string())

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    

Implement the Direction class:

In [0]:
#Reference:[1B] 
class Direction:
    """A direction class for agents that want to move in a 2D plane
        Usage:
            d = Direction("down")
            To change directions:
            d = d + "right" or d = d + Direction.R #Both do the same thing
            Note that the argument to __add__ must be a string and not a Direction object.
            Also, it (the argument) can only be right or left."""

    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __add__(self, heading):
        """
        >>> d = Direction('right')
        >>> l1 = d.__add__(Direction.L)
        >>> l2 = d.__add__(Direction.R)
        >>> l1.direction
        'up'
        >>> l2.direction
        'down'
        >>> d = Direction('down')
        >>> l1 = d.__add__('right')
        >>> l2 = d.__add__('left')
        >>> l1.direction == Direction.L
        True
        >>> l2.direction == Direction.R
        True
        """
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading, None)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading, None)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading, None)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading, None)

    def move_forward(self, from_location):
        """
        >>> d = Direction('up')
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (0, -1)
        >>> d = Direction(Direction.R)
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (1, 0)
        """
        # get the iterable class to return
        iclass = from_location.__class__
        x, y = from_location
        if self.direction == self.R:
            return iclass((x + 1, y))
        elif self.direction == self.L:
            return iclass((x - 1, y))
        elif self.direction == self.U:
            return iclass((x, y - 1))
        elif self.direction == self.D:
            return iclass((x, y + 1))

Set up the specifics of a 2D, grid-based environment:

In [0]:
#Reference:[1B] 
# modified from book
class XYEnvironment(Environment):
    """This class is for environments on a 2D plane, with locations
    labelled by (x, y) points, either discrete or continuous.
    Agents perceive things within a radius. Each agent in the
    environment has a .location slot which should be a location such
    as (0, 1), and a .holding slot, which should be a list of things
    that are held."""

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width, self.height)

    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """Return all things within radius of location."""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        return [(thing, radius2 - distance_squared(location, thing.location))
                for thing in self.things if distance_squared(
                location, thing.location) <= radius2]

    def percept(self, agent):
        """By default, agent perceives things within a default radius."""
        return self.things_near(agent.location)

 #MD: removed the 'release' option as not required
    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))    
        elif action == 'Grab':
            things = [thing for thing in self.list_things_at(agent.location)
                    if agent.can_grab(thing)]
            if things:
                agent.holding.append(things[0])
        #------------------- 
        #elif action == 'Release':
        #    if agent.holding:
        #        agent.holding.pop()
        #-------------------
    def default_location(self, thing):
        location = self.random_location_inbounds()
        while self.some_things_at(location, tree):
            # we will find a random location with no trees
            location = self.random_location_inbounds()
        return location

    def move_to(self, thing, destination):
        """Move a thing to a new location. Returns True on success or False if there is an tree.
        If thing is holding anything, they move with him."""
        thing.bump = self.some_things_at(destination, tree)
        if not thing.bump:
            thing.location = destination
            for o in self.observers:
                o.thing_moved(thing)
            for t in thing.holding:
                self.delete_thing(t)
                self.add_thing(t, destination)
                t.location = destination
        return thing.bump

    def add_thing(self, thing, location=None, exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if location is None:
            super().add_thing(thing)
        elif self.is_inbounds(location):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            super().add_thing(thing, location)

    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while location == exclude:
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

#MD: likely won't need this but leaving for now
    def delete_thing(self, thing):
        """Deletes thing, and everything it is holding (if thing is an agent)"""
        if isinstance(thing, Agent):
            for obj in thing.holding:
                super().delete_thing(obj)
                for obs in self.observers:
                    obs.thing_deleted(obj)

        super().delete_thing(thing)
        for obs in self.observers:
            obs.thing_deleted(thing)

    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        for x in range(self.width):
            self.add_thing(Wall(), (x, 0))
            self.add_thing(Wall(), (x, self.height - 1))
        for y in range(1, self.height - 1):
            self.add_thing(Wall(), (0, y))
            self.add_thing(Wall(), (self.width - 1, y))

        # Updates iteration start and end (with walls).
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width - 1, self.height - 1)

    
    def add_observer(self, observer):
        """Adds an observer to the list of observers.
        An observer is typically an EnvGUI.
        Each observer is notified of changes in move_to and add_thing,
        by calling the observer's methods thing_moved(thing)
        and thing_added(thing, loc)."""
        self.observers.append(observer)

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)

Define the specific Things which reside in the Pandemic environment:

In [0]:
#Reference:[1B] 
# modified from book
class Obstacle(Thing):
    """Something that can cause a bump, preventing an agent from
    moving into the same square it's in."""
    pass

class Tree(Obstacle):
    pass    

class Wall(Obstacle):
    pass

class Cure(Thing):
    def __eq__(self, rhs):        
        return rhs.__class__ == Cure
    pass

class Bump(Thing):
    pass

class Sunshine(Thing):
    pass

class Shadow(Thing):
    pass

class Smog(Thing):
    pass

class Hospital(Thing):  
    pass  

class Resident(Thing):  
    pass    

The PatientZero agent. This is the agent which can infect other agents.

In [0]:
#Reference:[1B] 
# modified from book
class PatientZero(Agent):
    
    #MD: Give the agent the ability to move
    # static for now
    #MD: if this agent can move, does the Smog move with him? Try the holding[smog] approach...
    holding = []
    sick = True

    def give_sick(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.health -= 10
            other.sick=True
            #MD: going to leave the self-health off for now to keep the game more interesting
            #self.health -= 5
    
    #MD: add the 'program' once ready to have PZ move around
    

Create the Agent which can perform logical inference and use the A* search method to plan it's route.

In [0]:
#Reference:[1B] 
# modified from book
class HybridPandemicAgent(Agent):
    """
    [Figure 7.20]
    An agent for the Pandemic world that does logical inference. 
    """

    def __init__(self, dimensions):
        self.dimrow = dimensions
        #MD: Make the change to FOL KB here:
        self.kb = PandemicKB(self.dimrow)
        self.t = 0
        self.plan = list()
        self.current_position = PatientZeroPosition(1, 1, 'UP')
        super().__init__(self.execute)

    def execute(self, percept):
        self.kb.make_percept_sentence(percept, self.t)
        self.kb.add_temporal_sentences(self.t)

        temp = list()

        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                if self.kb.ask_if_true(location(i, j, self.t)):
                    temp.append(i)
                    temp.append(j)

        if self.kb.ask_if_true(facing_north(self.t)):
            self.current_position = PatientZeroPositionPosition(temp[0], temp[1], 'UP')
        elif self.kb.ask_if_true(facing_south(self.t)):
            self.current_position =PatientZeroPosition(temp[0], temp[1], 'DOWN')
        elif self.kb.ask_if_true(facing_west(self.t)):
            self.current_position = PatientZeroPosition(temp[0], temp[1], 'LEFT')
        elif self.kb.ask_if_true(facing_east(self.t)):
            self.current_position = PatientZeroPosition(temp[0], temp[1], 'RIGHT')

        safe_points = list()
        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                if self.kb.ask_if_true(ok_to_move(i, j, self.t)):
                    safe_points.append([i, j])
#MD: if the percept is "sunshine", grab the cure
        if self.kb.ask_if_true(percept_sunshine(self.t)):
            goals = list()
            goals.append([1, 1])
            self.plan.append('Grab')
            actions = self.plan_route(self.current_position, goals, safe_points) #MD: then plan the route to the hospital
            self.plan.extend(actions)
            self.plan.append('Finish')

        if len(self.plan) == 0:
            unvisited = list()
            for i in range(1, self.dimrow + 1):
                for j in range(1, self.dimrow + 1):
                    for k in range(self.t):
                        if self.kb.ask_if_true(location(i, j, k)):
                            unvisited.append([i, j])
            unvisited_and_safe = list()
            for u in unvisited:
                for s in safe_points:
                    if u not in unvisited_and_safe and s == u:
                        unvisited_and_safe.append(u)

            temp = self.plan_route(self.current_position, unvisited_and_safe, safe_points)
            self.plan.extend(temp)

        if len(self.plan) == 0:
            not_unsafe = list()
            for i in range(1, self.dimrow + 1):
                for j in range(1, self.dimrow + 1):
                    if not self.kb.ask_if_true(ok_to_move(i, j, self.t)):
                        not_unsafe.append([i, j])
            temp = self.plan_route(self.current_position, not_unsafe, safe_points) #MD: Plan the route based upon what the agent has perceived
            self.plan.extend(temp)

        if len(self.plan) == 0:
            start = list()
            start.append([1, 1])
            temp = self.plan_route(self.current_position, start, safe_points) #MD: Plan the route based upon what the agent has perceived
            self.plan.extend(temp)
            self.plan.append('Finish')

        action = self.plan[0]
        self.plan = self.plan[1:]
        self.kb.make_action_sentence(action, self.t)
        self.t += 1

        return action

    def plan_route(self, current, goals, allowed):
        problem = PlanRoute(current, goals, allowed, self.dimrow)
        return astar_search(problem).solution()

The Explorer (player) agent.

In [0]:
#Reference:[1B] 
# modified from book
#class Explorer(Agent):
class Explorer(HybridPandemicAgent):
    holding = []
    killed_by = ""
    sick = False
    # MD: initially facing right
    direction = Direction("right")

    def can_grab(self, thing):
        """Explorer can only grab the cure"""
        return thing.__class__ == Cure

    def step(self):
        self.move()
        if self.sick == True:
            self.give_sick()
        if self.health == 0:
            self.alive = False    

#MD: TODO: The sample program subclasses the Explorer to incorporate
# the knowledgebase. Determine if this is the best approach, or just
# build that functionality into the Explorer class.        

The Resident agents. These are harmless until they are "sick"; they can then infect other agents (including player).

In [0]:
#AGENT
class Resident(Agent):
    """ An agent with fixed initial health."""
 # MD: TODO:review this in context once the environment is up and running.
 # can be refined; also need to stop the agent once health is 0   
    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        #possible_steps = self.model.grid.get_neighborhood(
        #    self.pos)

        #the agent moves about at random  
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_sick(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.health -= 10
            other.sick=True
            #MD: with every move, the resident agent gets sicker
            self.health -= 5

    def step(self):
        self.move()
        if self.health < 100:
            self.give_sick()
        if self.health == 0:
            self.alive = False  
            self.move = False  

Create the specific Pandemic environment:

In [0]:
#Reference:[1B] 
# modified from book
# Create the specific environment
class World(XYEnvironment):
    Tree_probability = 0.2  # Probability to spawn a tree in a location. (From Chapter 7.2)
    #Resident_probability = 0.2

    # # Room should be 4x4 grid of rooms. The extra 2 for walls
    #MD: TODO: make larger once simulation is running
    #MD: Calling it when running below
    def __init__(self, agent_program, width=6, height=6):
        super().__init__(width, height)
        self.init_world(agent_program)

    def init_world(self, program):
        """Spawn items in the world based on probabilities from the book"""

        "WALLS"
        self.add_walls()

        "TREES"
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if random.random() < self.Tree_probability:
                    self.add_thing(Tree(), (x, y), True)
                    self.add_thing(Shadow(), (x - 1, y), True)
                    self.add_thing(Shadow(), (x, y - 1), True)
                    self.add_thing(Shadow(), (x + 1, y), True)
                    self.add_thing(Shadow(), (x, y + 1), True)

        "PATIENT ZERO"
        w_x, w_y = self.random_location_inbounds(exclude=(1, 1))
        self.add_thing(PatientZero(lambda x: ""), (w_x, w_y), True)
        self.add_thing(Smog(), (w_x - 1, w_y), True)
        self.add_thing(Smog(), (w_x + 1, w_y), True)
        self.add_thing(Smog(), (w_x, w_y - 1), True)
        self.add_thing(Smog(), (w_x, w_y + 1), True)

        "CURE"
        self.add_thing(Cure(), self.random_location_inbounds(exclude=(1, 1)), True)

        "HOSPITAL"
        self.add_thing(Hospital(), self.random_location_inbounds(exclude=(1, 1)), True)

        "EXPLORER AGENT"
        self.add_thing(Explorer(program), (1, 1), True)

        #MD: adding 'program' from the agent superclass
        "RESIDENTS"
        self.add_thing(Resident(program), self.random_location_inbounds(exclude=(1, 1)), True)
        #self.add_thing(Resident(program), self.random_location_inbounds(exclude=(1, 1)), True)
        #self.add_thing(Resident(program), self.random_location_inbounds(exclude=(1, 1)), True)

        #"RESIDENTS"
        #for x in range(self.x_start, self.x_end):
        #    for y in range(self.y_start, self.y_end):
        #        if random.random() < self.Resident_probability:
        #            self.add_thing(Resident(), (x, y), True)

    def get_world(self, show_walls=True):
        """Return the items in the world"""
        result = []
        x_start, y_start = (0, 0) if show_walls else (1, 1)

        if show_walls:
            x_end, y_end = self.width, self.height
        else:
            x_end, y_end = self.width - 1, self.height - 1

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    def percepts_from(self, agent, location, tclass=Thing):
        """Return percepts from a given location,
        and replaces some items with percepts from chapter 7."""
        thing_percepts = {
            Cure: Sunshine(),
            Wall: Bump(),
            
            PatientZero: Smog(),
            Tree: Shadow()}

        """Agents don't need to get their percepts"""
        thing_percepts[agent.__class__] = None

        """The sunshine percept is only available in the same cell as the cure"""
        if location != agent.location:
            thing_percepts[Cure] = None

        result = [thing_percepts.get(thing.__class__, thing) for thing in self.things
                  if thing.location == location and isinstance(thing, tclass)]
        return result if len(result) else [None]

    def percept(self, agent):
        """Return things in adjacent (not diagonal) cells of the agent.
        Result format: [Left, Right, Up, Down, Current location]"""
        x, y = agent.location
        result = []
        result.append(self.percepts_from(agent, (x - 1, y)))
        result.append(self.percepts_from(agent, (x + 1, y)))
        result.append(self.percepts_from(agent, (x, y - 1)))
        result.append(self.percepts_from(agent, (x, y + 1)))
        result.append(self.percepts_from(agent, (x, y)))

        return result

    def execute_action(self, agent, action):
        """Modify the state of the environment based on the agent's actions.
        Performance score taken directly out of the book."""

        if isinstance(agent, Explorer) and self.in_danger(agent):
            return

        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
            agent.performance -= 1
        elif action == 'TurnLeft':
            agent.direction += Direction.L
            agent.performance -= 1
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
            agent.performance -= 1
        elif action == 'Grab':
            things = [thing for thing in self.list_things_at(agent.location)
                      if agent.can_grab(thing)]
            if len(things):
                print("Grabbing", things[0].__class__.__name__)
                if len(things):
                    agent.holding.append(things[0])
            agent.performance -= 1
        elif action == 'Finish':
          #MD: TODO: This needs to be changed to the hospital location
            if agent.location == (1, 1):  # Agent can only finish in (1,1) in WW
                agent.performance += 1000 if CURE() in agent.holding else 0
                self.delete_thing(agent)
          
#MD: shouldn't need this; but needs to be removed carefully
    def in_danger(self, agent):
 #MD: May have to switch the sub-return back to False       
        for thing in self.list_things_at(agent.location):
            if isinstance(thing, Tree):
                agent.alive = True
                agent.performance -= 5
                agent.health -= 10
                #agent.killed_by = thing.__class__.__name__
                return True
        return False

#MD: May have to switch the sub-return back to False
        for thing in self.list_things_at(agent.location):
            if (isinstance(thing, PatientZero) and thing.alive):
                agent.alive = True
                agent.performance -= 50
                agent.health -= 50
                agent.killed_by = thing.__class__.__name__
                return True
        return False

    def is_done(self):
        """The game is over when the Explorer is killed
        or if he finishes at hospital (NOT AT) at (1,1)."""
        #MD: TODO: The explorer needs to locate the hospital via A* algorithm
        explorer = [agent for agent in self.agents if isinstance(agent, Explorer)]
        if len(explorer):
            if explorer[0].alive:
                return False
            else:
                print("Explorer died {} [LOSE].".format(explorer[0].killed_by))
        else:
            print("Explorer Finished at hospital {}."
                  .format("with Cure [WIN]!" if Cure() not in self.things else "without Cure [+0]"))
        return True


NameError: ignored

In [0]:
#KNOWLEDGEBASE
# Convert to Conjunctive Normal Form (CNF)

# REMOVED: snippet_3


Knowledgebase structure:

In [0]:
#Reference:[1B] 
# modified from book
class KB:
    """A knowledge base to which you can tell and ask sentences.
    To create a KB, first subclass this class and implement
    tell, ask_generator, and retract. Why ask_generator instead of ask?
    The book is a bit vague on what ask means --
    For a Propositional Logic KB, ask(P & Q) returns True or False, but for an
    FOL KB, something like ask(Brother(x, y)) might return many substitutions
    such as {x: Cain, y: Abel}, {x: Abel, y: Cain}, {x: George, y: Jeb}, etc.
    So ask_generator generates these one at a time, and ask either returns the
    first one or returns False."""

    def __init__(self, sentence=None):
        if sentence:
            self.tell(sentence)

    def tell(self, sentence):
        """Add the sentence to the KB."""
        raise NotImplementedError

    def ask(self, query):
        """Return a substitution that makes the query true, or, failing that, return False."""
        return first(self.ask_generator(query), default=False)

    def ask_generator(self, query):
        """Yield all the substitutions that make query true."""
        raise NotImplementedError

    def retract(self, sentence):
        """Remove sentence from the KB."""
        raise NotImplementedError

Propositional Knowledgebase:

In [0]:
#KNOWLEDGEBASE
#REMOVED: snippet_4


A generic logical knowledge-based agent program.

In [0]:
#Reference:[1B] 
# modified from book
#MD: Used by FOLKB
#MD: This is causing the problem
#MD: NOTE: this is the location of the problem, not "causing" it 
def KBAgentProgram(kb):
    """
    [Figure 7.1 from the book]
    function KB-AGENT(percept ) returns an action
      persistent: KB, a knowledge base
          t , a counter, initially 0, indicating time
      TELL(KB,MAKE-PERCEPT-SENTENCE(percept , t ))
      action ←ASK(KB,MAKE-ACTION-QUERY(t ))
      TELL(KB,MAKE-ACTION-SENTENCE(action, t ))
      t ←t + 1
      return action
    """
   
    steps = itertools.count()

    def program(percept):
        t = next(steps)
        kb.tell(make_percept_sentence(percept, t))
        action = kb.ask(make_action_query(t))
        kb.tell(make_action_sentence(action, t))
        return action

    def make_percept_sentence(percept, t):
        return Expr('Percept')(percept, t)

    def make_action_query(t):
        return expr('ShouldDo(action, {})'.format(t))

    def make_action_sentence(action, t):
        return Expr('Did')(action[expr('action')], t)

    return program


def is_symbol(s):
    """A string s is a symbol if it starts with an alphabetic char.
    >>> is_symbol('R2D2')
    True
    """
    return isinstance(s, str) and s[:1].isalpha()


def is_var_symbol(s):
    """A logic variable symbol is an initial-lowercase string.
    >>> is_var_symbol('EXE')
    False
    """
    return is_symbol(s) and s[0].islower()


def is_prop_symbol(s):
    """A proposition logic symbol is an initial-uppercase string.
    >>> is_prop_symbol('exe')
    False
    """
    return is_symbol(s) and s[0].isupper()


def variables(s):
    """Return a set of the variables in expression s.
    >>> variables(expr('F(x, x) & G(x, y) & H(y, z) & R(A, z, 2)')) == {x, y, z}
    True
    """
    return {x for x in subexpressions(s) if is_variable(x)}


def is_definite_clause(s): 
    """Returns True for exprs s of the form A & B & ... & C ==> D,
    where all literals are positive. In clause form, this is
    ~A | ~B | ... | ~C | D, where exactly one clause is positive.
    >>> is_definite_clause(expr('Farmer(Mac)'))
    True
    """
#MD:SENTENCEERROR: Sets up the failure.    
    if is_symbol(s.op):
        return True
    elif s.op == '==>':
        antecedent, consequent = s.args
        return is_symbol(consequent.op) and all(is_symbol(arg.op) for arg in conjuncts(antecedent))
    else:
        return False


def parse_definite_clause(s):
    """Return the antecedents and the consequent of a definite clause."""
    assert is_definite_clause(s)
    if is_symbol(s.op):
        return [], s
    else:
        antecedent, consequent = s.args
        return conjuncts(antecedent), consequent
#Z, R, T, S, M = map(Expr, 'Z','B','T','S','M')
Z, R, T, S, M = map(Expr, 'ZBTSM')
#A, B, C, D, E, F, G, P, Q, a, x, y, z, u = map(Expr, 'ABCDEFGPQaxyzu')        

*TODO: Need to test here for the errors.*

In [0]:
is_definite_clause(expr('~Z(1, 1)'))
#MD: The statement causing the error is saying "PatientZero is not in 1,1", which is valid. 
# The problem is this is not in the "antecedent, consequent = s.args" format.
# It can't "TELL" the FolKB (below) because of the formatting;
# also, ~Z(1, 1) is a setup statement, not something to infer.


False

In [0]:
#MD: may need to paste the Unify module here

The First Order Logic base from the book. NOTE: Left some unrelated items in for reference

In [0]:
#Reference:[1B] 
# modified from book
class FolKB(KB):
    """A knowledge base consisting of first-order definite clauses.
    >>> kb0 = FolKB([expr('Farmer(Mac)'), expr('Rabbit(Pete)'),
    ...              expr('(Rabbit(r) & Farmer(f)) ==> Hates(f, r)')])
    >>> kb0.tell(expr('Rabbit(Flopsie)'))
    >>> kb0.retract(expr('Rabbit(Pete)'))
    >>> kb0.ask(expr('Hates(Mac, x)'))[x]
    Flopsie
    >>> kb0.ask(expr('Wife(Pete, x)'))
    False
    """

    def __init__(self, clauses=None):
        super().__init__()
        self.clauses = []  # inefficient: no indexing
        if clauses:
            for clause in clauses:
                self.tell(clause)

#MD:SENTENCEERROR: Triggering the fail when trying to run.  
    def tell(self, sentence):
        if is_definite_clause(sentence):
            self.clauses.append(sentence)
        #MD:SENTENCEERROR: For now, going to comment out the error to try to get the program to run.
        #Another option is to specify the location of PZ & trees and remove the 'tell' statements. 
        else:
            raise Exception('Not a definite clause: {}'.format(sentence)) #HERE

    def ask_generator(self, query):
        return fol_bc_ask(self, query)

    def retract(self, sentence):
        self.clauses.remove(sentence)

    def fetch_rules_for_goal(self, goal):
        return self.clauses


def fol_fc_ask(kb, alpha):
    """
    [Figure 9.3]
    A simple forward-chaining algorithm.
    """
    # TODO: improve efficiency
    kb_consts = list({c for clause in kb.clauses for c in constant_symbols(clause)})

    def enum_subst(p):
        query_vars = list({v for clause in p for v in variables(clause)})
        for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
            theta = {x: y for x, y in zip(query_vars, assignment_list)}
            yield theta

    # check if we can answer without new inferences
    for q in kb.clauses:
        phi = unify_mm(q, alpha)
        if phi is not None:
            yield phi

    while True:
        new = []
        for rule in kb.clauses:
            p, q = parse_definite_clause(rule)
            for theta in enum_subst(p):
                if set(subst(theta, p)).issubset(set(kb.clauses)):
                    q_ = subst(theta, q)
                    if all([unify_mm(x, q_) is None for x in kb.clauses + new]):
                        new.append(q_)
                        phi = unify_mm(q_, alpha)
                        if phi is not None:
                            yield phi
        if not new:
            break
        for clause in new:
            kb.tell(clause)
    return None


def fol_bc_ask(kb, query):
    """
    [Figure 9.6]
    A simple backward-chaining algorithm for first-order logic.
    KB should be an instance of FolKB, and query an atomic sentence.
    """
    return fol_bc_or(kb, query, {})


def fol_bc_or(kb, goal, theta):
    for rule in kb.fetch_rules_for_goal(goal):
        lhs, rhs = parse_definite_clause(standardize_variables(rule))
        for theta1 in fol_bc_and(kb, lhs, unify_mm(rhs, goal, theta)):
            yield theta1


def fol_bc_and(kb, goals, theta):
    if theta is None:
        pass
    elif not goals:
        yield theta
    else:
        first, rest = goals[0], goals[1:]
        for theta1 in fol_bc_or(kb, subst(theta, first), theta):
            for theta2 in fol_bc_and(kb, rest, theta1):
                yield theta2




In [0]:
#NOT IN USE
# Another implementation of FolKB
# REMOVED: snippet_5

Functions to convert into Expr format:

In [0]:
#Reference:[1B] 
# modified from book
# Expr functions for patient_zeroKB and Hybridpatient_zeroAgent

def facing_east(time):
    return Expr('FacingEast', time)


def facing_west(time):
    return Expr('FacingWest', time)


def facing_north(time):
    return Expr('FacingNorth', time)


def facing_south(time):
    return Expr('FacingSouth', time)


def patient_zero(x, y):
    return Expr('Z', x, y)

def resident(x, y):
    return Expr('R', x, y)    

def tree(x, y):
    return Expr('T', x, y)


def shadow(x, y):
    return Expr('S', x, y)


def smog(x, y):
    return Expr('M', x, y)


def patient_zero_alive(time):
    return Expr('patientZeroAlive', time)


def percept_smog(time):
    return Expr('Smog', time)


def percept_shadow(time):
    return Expr('Shadow', time)


def percept_sunshine(time):
    return Expr('Sunshine', time)


def percept_bump(time):
    return Expr('Bump', time)


def move_forward(time):
    return Expr('Forward', time)


def turn_left(time):
    return Expr('TurnLeft', time)


def turn_right(time):
    return Expr('TurnRight', time)


def ok_to_move(x, y, time):
    return Expr('OK', x, y, time)


def location(x, y, time=None):
    if time is None:
        return Expr('L', x, y)
    else:
        return Expr('L', x, y, time)


# Symbols

def implies(lhs, rhs):
    return Expr('==>', lhs, rhs)


def equiv(lhs, rhs):
    return Expr('<=>', lhs, rhs)


# Helper Function

def new_disjunction(sentences):
    t = sentences[0]
    for i in range(1, len(sentences)):
        t |= sentences[i]
    return t

The program-specific Knowledgebase:

In [0]:
#Reference:[1B] 
#class PandemicKB(PropKB):
class PandemicKB(FolKB):
    """
    Create a Knowledge Base that contains the KB information which is not able to be inferred, with time zero.
    """

    def __init__(self, dimrow):
        super().__init__()
        self.dimrow = dimrow
        ###############################################################################################
        #MD: There is no patientZero or tree in (1,1)
        #MD:SENTENCEERROR: self.tell(~patient_zero(1, 1)) is being fed into the KB 
        #which is causing the error, as it is not an atomic sentence.
        #MD: Going to comment this out now, to try to move forward
        #MD: This information needs to go into the KB however.
        ###############################################################################################
        self.tell(~patient_zero(1, 1))
        self.tell(~tree(1, 1))


        for y in range(1, dimrow + 1):
            for x in range(1, dimrow + 1):

                tree_in = list()
                patient_zero_in = list()

                if x > 1:  # West room exists
                    tree_in.append(tree(x - 1, y))
                    patient_zero_in.append(patient_zero(x - 1, y))

                if y < dimrow:  # North room exists
                    tree_in.append(tree(x, y + 1))
                    patient_zero_in.append(patient_zero(x, y + 1))

                if x < dimrow:  # East room exists
                    tree_in.append(tree(x + 1, y))
                    patient_zero_in.append(patient_zero(x + 1, y))

                if y > 1:  # South room exists
                    tree_in.append(tree(x, y - 1))
                    patient_zero_in.append(patient_zero(x, y - 1))

                self.tell(equiv(shadow(x, y), new_disjunction(tree_in)))
                self.tell(equiv(smog(x, y), new_disjunction(patient_zero_in)))

        # Rule that describes existence of at least one patient_zero
        patient_zero_at_least = list()
        for x in range(1, dimrow + 1):
            for y in range(1, dimrow + 1):
                patient_zero_at_least.append(patient_zero(x, y))

        self.tell(new_disjunction(patient_zero_at_least))

        # Rule that describes existence of at most one patient_zero
        for i in range(1, dimrow + 1):
            for j in range(1, dimrow + 1):
                for u in range(1, dimrow + 1):
                    for v in range(1, dimrow + 1):
                        if i != u or j != v:
                            self.tell(~patient_zero(i, j) | ~patient_zero(u, v))

        # Temporal rules at time zero
        self.tell(location(1, 1, 0))
        for i in range(1, dimrow + 1):
            for j in range(1, dimrow + 1):
                self.tell(implies(location(i, j, 0), equiv(percept_shadow(0), shadow(i, j))))
                self.tell(implies(location(i, j, 0), equiv(percept_smog(0), smog(i, j))))
                if i != 1 or j != 1:
                    self.tell(~location(i, j, 0))

        self.tell(patient_zero_alive(0))
        self.tell(facing_east(0))
        self.tell(~facing_north(0))
        self.tell(~facing_south(0))
        self.tell(~facing_west(0))


    def make_action_sentence(self, action, time):
        actions = [move_forward(time),  turn_left(time), turn_right(time)] #MD: Remove the 'shoot' component ; was 2nd in list [shoot(time),]

        for a in actions:
            if action is a:
                self.tell(action)
            else:
                self.tell(~a)

    def make_percept_sentence(self, percept, time):
        # Sunshine, Bump, smog, shadow
        flags = [0, 0, 0, 0]

        # Things perceived
        if isinstance(percept, Sunshine):
            flags[0] = 1
            self.tell(percept_sunshine(time))
        elif isinstance(percept, Bump):
            flags[1] = 1
            self.tell(percept_bump(time))
        elif isinstance(percept, smog):
            flags[2] = 1
            self.tell(percept_smog(time))
        elif isinstance(percept, shadow):
            flags[3] = 1
            self.tell(percept_shadow(time))
        

        # Things not perceived
        for i in range(len(flags)):
            if flags[i] == 0:
                if i == 0:
                    self.tell(~percept_sunshine(time))
                elif i == 1:
                    self.tell(~percept_bump(time))
                elif i == 2:
                    self.tell(~percept_smog(time))
                elif i == 3:
                    self.tell(~percept_shadow(time))
                

    def add_temporal_sentences(self, time):
        if time == 0:
            return
        t = time - 1

        # current location rules
        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                self.tell(implies(location(i, j, time), equiv(percept_shadow(time), shadow(i, j))))
                self.tell(implies(location(i, j, time), equiv(percept_smog(time), smog(i, j))))
                s = list()
                s.append(equiv(location(i, j, time), location(i, j, time) & ~move_forward(time) | percept_bump(time)))
                if i != 1:
                    s.append(location(i - 1, j, t) & facing_east(t) & move_forward(t))
                if i != self.dimrow:
                    s.append(location(i + 1, j, t) & facing_west(t) & move_forward(t))
                if j != 1:
                    s.append(location(i, j - 1, t) & facing_north(t) & move_forward(t))
                if j != self.dimrow:
                    s.append(location(i, j + 1, t) & facing_south(t) & move_forward(t))

                # add sentence about location i,j
                self.tell(new_disjunction(s))

                # add sentence about safety of location i,j
                self.tell(equiv(ok_to_move(i, j, time), ~tree(i, j) & ~patient_zero(i, j) & patient_zero_alive(time)))

        # Rules about current orientation

        a = facing_north(t) & turn_right(t)
        b = facing_south(t) & turn_left(t)
        c = facing_east(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_east(time), a | b | c)
        self.tell(s)

        a = facing_north(t) & turn_left(t)
        b = facing_south(t) & turn_right(t)
        c = facing_west(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_west(time), a | b | c)
        self.tell(s)

        a = facing_east(t) & turn_left(t)
        b = facing_west(t) & turn_right(t)
        c = facing_north(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_north(time), a | b | c)
        self.tell(s)

        a = facing_west(t) & turn_left(t)
        b = facing_east(t) & turn_right(t)
        c = facing_south(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_south(time), a | b | c)
        self.tell(s)

        # Rules about last action
        self.tell(equiv(move_forward(t), ~turn_right(t) & ~turn_left(t)))

        # Rule about patient_zero (dead or alive)
        self.tell(equiv(patient_zero_alive(time), patient_zero_alive(t)))

    def ask_if_true(self, query):
        return pl_resolution(self, query)

In [0]:
#Reference:[1B] 
#MD: TODO: Need to clone this for the Residne agent position
class PatientZeroPosition:
    def __init__(self, x, y, orientation):
        self.X = x
        self.Y = y
        self.orientation = orientation

    def get_location(self):
        return self.X, self.Y

    def set_location(self, x, y):
        self.X = x
        self.Y = y

    def get_orientation(self):
        return self.orientation

    def set_orientation(self, orientation):
        self.orientation = orientation

    def __eq__(self, other):
        if other.get_location() == self.get_location() and other.get_orientation() == self.get_orientation():
            return True
        else:
            return False

The superclass of the PlanRoute problem:

In [0]:
#Reference:[1B] 
# modified from book
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError


Adding node class for the search algorithms

*NOTE: Override this with the new Node class below...

In [0]:
##########################
# NOTE: There is an alternate
# Node class below; experiment with that
##########################
#Reference:[1B] 

class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what we
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)


In [0]:
# https://gist.githubusercontent.com/Nicholas-Swift/003e1932ef2804bebef2710527008f44/raw/8fd79d81dcd5b52d01918b29689e0727154f886c/astar.py
# MD: can put the Hospital location into a variable and use the algorithm above to solve the issue.
# Would also solve the finishing problem

Implementing abstract problem solving Program class (for when the cure is located)

In [0]:
#Reference:[1B] 
# modified from book
#MD: No longer required as we are using the HybridPandemicAgent

class SimpleProblemSolvingAgentProgram:
    """
    [Figure 3.1]
    Abstract framework for a problem-solving agent.
    """

    def __init__(self, initial_state=None):
        """State is an abstract representation of the state
        of the world, and seq is the list of actions required
        to get to a particular state from the initial state(root)."""
        self.state = initial_state
        self.seq = []

    def __call__(self, percept):
        """[Figure 3.1] Formulate a goal and problem, then
        search for a sequence of actions to solve it."""
        self.state = self.update_state(self.state, percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq = self.search(problem)
            if not self.seq:
                return None
        return self.seq.pop(0)

    def update_state(self, state, percept):
        raise NotImplementedError

    def formulate_goal(self, state):
        raise NotImplementedError

    def formulate_problem(self, state, goal):
        raise NotImplementedError

    def search(self, problem):
        raise NotImplementedError



Best first graph search methods (used by A-Star algorithm)

In [0]:
#Reference:[1B] 
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

A* Algorithm:

In [0]:
#Reference:[1B] 
##########################
# NOTE: There is an alternate
# A* search below; experiment with that
##########################
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

########## MODIFIED SEARCH START ##########

In [0]:
#Reference:[3] 
##########################
# NOTE: Don't run until details are worked out
##########################
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

###################################
# add original Node class details
# and put here:       
###################################

In [0]:
#Reference:[3] 
##########################
# NOTE: Don't run until details are worked out
##########################
def astar_search(maze, start, end):
#def astar_search(World, start, end):
    

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)

In [0]:
#Reference:[3] 
# Can't test this until the initialization problem is fixed
# TEST PROGRAM

# UNCOMMENT THE BELOW: 
#    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    start = (0, 0)
    end = (7, 6)
    path = astar(maze, start, end)   
#---------------------------------------------   
    
    #start = Explorer.location()
    #end = Hospital.location()
    #path = astar(World, start, end)
    
    print(path)

######### MODIFIED SEARCH END ##########

Used to plan the route to the hospital once the cure is acquired:

In [0]:
#Reference:[1B] 
class PlanRoute(Problem):
    """ The problem of moving the Hybrid Pandemic Agent from one place to other """

    def __init__(self, initial, goal, allowed, dimrow):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)
        self.dimrow = dimrow
        self.goal = goal
        self.allowed = allowed

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only three possible actions
        in any given state of the environment """

        possible_actions = ['Forward', 'TurnLeft', 'TurnRight']
        x, y = state.get_location()
        orientation = state.get_orientation()

        # Prevent Bumps
        if x == 1 and orientation == 'LEFT':
            if 'Forward' in possible_actions:
                possible_actions.remove('Forward')
        if y == 1 and orientation == 'DOWN':
            if 'Forward' in possible_actions:
                possible_actions.remove('Forward')
        if x == self.dimrow and orientation == 'RIGHT':
            if 'Forward' in possible_actions:
                possible_actions.remove('Forward')
        if y == self.dimrow and orientation == 'UP':
            if 'Forward' in possible_actions:
                possible_actions.remove('Forward')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """
        x, y = state.get_location()
        proposed_loc = list()

        # Move Forward
        if action == 'Forward':
            if state.get_orientation() == 'UP':
                proposed_loc = [x, y + 1]
            elif state.get_orientation() == 'DOWN':
                proposed_loc = [x, y - 1]
            elif state.get_orientation() == 'LEFT':
                proposed_loc = [x - 1, y]
            elif state.get_orientation() == 'RIGHT':
                proposed_loc = [x + 1, y]
            else:
                raise Exception('InvalidOrientation')

        # Rotate counter-clockwise
        elif action == 'TurnLeft':
            if state.get_orientation() == 'UP':
                state.set_orientation('LEFT')
            elif state.get_orientation() == 'DOWN':
                state.set_orientation('RIGHT')
            elif state.get_orientation() == 'LEFT':
                state.set_orientation('DOWN')
            elif state.get_orientation() == 'RIGHT':
                state.set_orientation('UP')
            else:
                raise Exception('InvalidOrientation')

        # Rotate clockwise
        elif action == 'TurnRight':
            if state.get_orientation() == 'UP':
                state.set_orientation('RIGHT')
            elif state.get_orientation() == 'DOWN':
                state.set_orientation('LEFT')
            elif state.get_orientation() == 'LEFT':
                state.set_orientation('UP')
            elif state.get_orientation() == 'RIGHT':
                state.set_orientation('DOWN')
            else:
                raise Exception('InvalidOrientation')

        if proposed_loc in self.allowed:
            state.set_location(proposed_loc[0], [proposed_loc[1]])

        return state

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """

        return state.get_location() == tuple(self.goal)

    def h(self, node):
        """ Return the heuristic value for a given state."""

        # Manhattan Heuristic Function
        x1, y1 = node.state.get_location()
        x2, y2 = self.goal
#MD:TODO: change the return function to implement A Star
        return abs(x2 - x1) + abs(y2 - y1)



iPythonBlocks graphics module

In [0]:
#Reference:[1B] 
class GraphicEnvironment(XYEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""
        super().__init__(width, height)
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)

    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))


In [0]:
#Reference:[1B] 
# modified from book
# Eventually get it to display with iPythonblocks:
color = {"Shadow": (255, 153, 51), #orange
        "Tree": (153,102,51), #brown
        "Cure": (253, 208, 23), #gold
        "Sunshine": (255, 255, 0),#yellow
        "PatientZero": (43, 27, 23), #black
        "Smog": (128, 128, 128), #gray
        "Explorer": (0, 0, 255), #blue
        "Wall": (44, 53, 57) #dk gray
        }

The simulation should be ready to run.
We have established the Agent types, searches to use, the knowledgebase, and the environment.

## **Run the simulation**

In [0]:
def program(percepts):
    '''Returns an action based on it's percepts'''
    print(percepts)
    return input()
    # This exists in the Agent program
    #return eval(input('Percept={}; action? '.format(percept)))

#w = World(program, 7, 7) 
#w = World(program, 5, 5)  
#w = World(HybridPandemicAgent)

w = World(program)

grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                grid[y, x] = color[world[x][y][-1].__class__.__name__]

def step():
    global grid, w
    draw_grid(w.get_world())
    grid.show()
    #print(self.env.to_string()
    w.step()

In [0]:
World.get_world(w)

Percepts queue order:


*   [Left, Right, Up, Down, Current location]



In [0]:
w.step()
# only runs 4 iterations at most...

In [0]:
#MD: Need to add printing so we know health with every 
# move, resident/ PZ encountered, etc. 



In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

Errors:


*   self.tell(~patient_zero(1, 1)) is being fed verbatim into the KB which is causing the error, as it is not an atomic sentence. This information needs to go into the KB however.
*   w = World(program, 5, 5) is throwing a TypeError: unsupported operand type(s) for +: 'function' and 'int'



